In [1]:
import torch
import torchvision.transforms as transforms
import numpy as np
import random
from torch.utils.data import Dataset, random_split, DataLoader
import pandas as pd
from PIL import Image
import os


DATASET_PATH = 'data/train'
TRAIN_CSV = 'data/train.csv'

# Fix the random seed for reproducibility
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark = False


In [2]:
# Convert labels to binary arrays
def encode_label(label):
    target = torch.zeros(30)
    for l in str(label).split(' '):
        target[int(l)] = 1.
    return target

def decode_target(target, text_labels=False, threshold=0.5):
    result = []
    for i, x in enumerate(target):
        if (x >= threshold):
            if text_labels:
                result.append(labels[i] + "(" + str(i) + ")")
            else:
                result.append(str(i))
    return ' '.join(result)

In [3]:
class HumanProteinDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.data = pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.data)    
    
    def __getitem__(self, idx):
        colors = ['red']
        row = self.data.loc[idx]
        img_id, img_label = row['Id'], row['Target']
        img = self.open_rgby(img_id)
#         img_fname = self.root_dir + "/" + str(img_id) +color + ".png"
#             img = Image.open(img_fname)
        if self.transform:
            img = self.transform(img)
        return img, encode_label(img_label)
    
    def open_rgby(self, id): #a function that reads RGBY image
        colors = ['red','green','blue','yellow']
        #flags = cv2.IMREAD_GRAYSCALE
        img = [Image.open( self.root_dir + "/" + str(id) + "_" +color + ".png")
               for color in colors]
        return np.stack(img, axis=-1)
    
#transform = transforms.Compoe([transforms.ToTensor()])
#dataset = HumanProteinDataset(root_dir, transform=transform)

In [4]:
def load_data(batch_size):
    data = HumanProteinDataset(
        csv_file = TRAIN_CSV,
        root_dir= DATASET_PATH, 
        transform=transforms.ToTensor()
    )
    # 10 percent of training data are utilized as a test set
    test_pct = 0.1
    test_size = int(test_pct * len(data))
    train_size = len(data) - test_size

    train_data, test_data = random_split(data, [train_size, test_size])
    
    train_dataloader = torch.utils.data.DataLoader(
        train_data, 
        batch_size=batch_size,
        shuffle=True, 
        num_workers=0
    )
    
    test_dataloader = torch.utils.data.DataLoader(
        test_data, 
        batch_size=batch_size,
        shuffle=True, 
        num_workers=4
    )
    return train_dataloader, test_dataloader

In [5]:
# Train NN
def test(test_dataloader, model):
    '''
    This function will test the model performance using testing data.
    DO NOT MODIFY!!
    '''
    with torch.no_grad():
        accu_number = 0.0
        for x, y in test_dataloader:
            x, y = x.to(device), y.to(device)
            predicted_class = torch.argmax(model(x),dim=1)
            accu_number += torch.sum(predicted_class == y)
        print('testing accuracy: %.4f' % (accu_number/len(test_dataloader.dataset)))
def train(dataloader, model, loss_fn, optimizer):
    '''
    This function will conduct one-epoch training.
    DO NOT MODIFY!!
    '''
    for x, y in dataloader:
        print("x")
        x, y = x.to(device), y.to(device)
        pred = model(x)
        print(pred)
        loss = loss_fn(pred, y)
        print("loss")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
class Network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(4, 32, kernel_size=4, stride =2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            
            torch.nn.Conv2d(32, 64, kernel_size=4, stride =2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            
            torch.nn.Conv2d(64, 128, kernel_size=4, stride =2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            
            torch.nn.Conv2d(128, 256, kernel_size=4, stride =2, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            
            torch.nn.Flatten(), 
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 30),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        pred = self.layers(x)
        return pred


    
    

In [6]:
def main(hyper_param):
    model = Network().to(device)
    train_dataloader, test_dataloader = load_data(hyper_param['batch_size'])
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = getattr(torch.optim, hyper_param['optimizer'])(
        model.parameters(), 
        **hyper_param['optim_param'] ## keyword unpacking
    )
    for t in range(hyper_param['n_epochs']):
        print(f"Epoch {t}", end=' ')
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyper-parameters
hyper_param = {
    'batch_size': 200,
    'n_epochs':5,
    'optimizer': 'SGD',
    'optim_param': {
        ## This dict should be changed according to the selection of optimizer ##
        'lr': 0.0001
    }
}

# define an instance of neuralnetworks
#main(model, hyper_param)
main(hyper_param)

Epoch 0 x


RuntimeError: CUDA out of memory. Tried to allocate 1.56 GiB (GPU 0; 4.00 GiB total capacity; 2.35 GiB already allocated; 0 bytes free; 2.37 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF